In [2]:
import cv2
import mediapipe as mp
import math
import time
from collections import deque
import traceback
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
try:
    from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
except ImportError:
    print("pycaw not available. Audio control features will be disabled.")
    AudioUtilities = None

# Initialize MediaPipe Hand module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

# Function to calculate distance between two points
def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Function to control music playback
def control_music(distance, min_distance=30, max_distance=300):
    if AudioUtilities is None:
        return 0
    
    try:
        devices = AudioUtilities.GetSpeakers()
        interface = devices.Activate(
            IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
        volume = cast(interface, POINTER(IAudioEndpointVolume))
        
        # Map distance to volume (0.0 to 1.0)
        if distance < min_distance:
            volume_level = 0.0
        elif distance > max_distance:
            volume_level = 1.0
        else:
            volume_level = (distance - min_distance) / (max_distance - min_distance)
        
        # Convert to decibels
        volume_db = volume_level * (volume.GetVolumeRange()[1] - volume.GetVolumeRange()[0]) + volume.GetVolumeRange()[0]
        volume.SetMasterVolumeLevel(volume_db, None)
        return volume_level
    except Exception as e:
        print(f"Error controlling audio: {e}")
        print(traceback.format_exc())
        return 0

# Function to draw volume bar
def draw_volume_bar(frame, volume_level):
    frame_height, frame_width = frame.shape[:2]
    bar_height = int(volume_level * frame_height)
    bar_width = 30
    bar_x = frame_width - 50  # 50 pixels from the right edge

    # Draw background bar (gray)
    cv2.rectangle(frame, (bar_x, 0), (bar_x + bar_width, frame_height), (200, 200, 200), -1)
    
    # Draw volume level (green)
    cv2.rectangle(frame, (bar_x, frame_height - bar_height), (bar_x + bar_width, frame_height), (0, 255, 0), -1)
    
    # Draw outline
    cv2.rectangle(frame, (bar_x, 0), (bar_x + bar_width, frame_height), (0, 0, 0), 2)

# Start capturing video
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video capture device.")
    exit()

current_window_name = "Hand Tracking"

# For smoothing
distance_queue = deque(maxlen=5)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            thumb_tip = hand_landmarks.landmark[4]
            index_finger_tip = hand_landmarks.landmark[8]
            
            h, w, c = frame.shape
            x1, y1 = int(thumb_tip.x * w), int(thumb_tip.y * h)
            x2, y2 = int(index_finger_tip.x * w), int(index_finger_tip.y * h)
            
            distance = calculate_distance(x1, y1, x2, y2)
            distance_queue.append(distance)
            avg_distance = sum(distance_queue) / len(distance_queue)

            cv2.putText(frame, f'({x1}, {y1})', (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(frame, f'({x2}, {y2})', (x2, y2), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
            
            cv2.line(frame, (x1, y1), (x2, y2), (0, 0, 255), 2, cv2.LINE_AA)
            
            mid_x, mid_y = (x1 + x2) // 2, (y1 + y2) // 2
            cv2.putText(frame, f'Dist: {avg_distance:.2f}', (mid_x, mid_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
            
            volume_level = control_music(avg_distance)
            cv2.putText(frame, f'Volume: {volume_level:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            
            # Draw the volume bar
            draw_volume_bar(frame, volume_level)

    else:
        # If no hand is detected, draw the volume bar at 0
        draw_volume_bar(frame, 0)

    cv2.imshow(current_window_name, frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [3]:
pip install --upgrade pycaw

Note: you may need to restart the kernel to use updated packages.
